In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 1. CNN model basics with PyTorch

## 1) Import libraries and define inputs

In [2]:
import torch
import torch.nn as nn

In [4]:
# define a random tensor
# batch, channel, height, width
inputs = torch.Tensor(1, 1, 28, 28)
print('Size of the tensor: {}'.format(inputs.shape))

Size of the tensor: torch.Size([1, 1, 28, 28])


## 2) Convolutional and pooling layers
---
- torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros', device=None, dtype=None)


In [5]:
# first layer
conv1 = nn.Conv2d(1, 32, 3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [7]:
# second layer
conv2 = nn.Conv2d(32, 64, 3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


Now, max pooling will be applied.
- torch.nn.MaxPool2d(kernel_size, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)

In [8]:
# pooling layer
pool = nn.MaxPool2d(2)    # when one integer given, it applies to both kernel size and stride
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


## 3) Models

In [9]:
out = conv1(inputs)
print('Input size: {}'.format(inputs.shape))
print('Conv1 output size: {}'.format(out.shape))

Input size: torch.Size([1, 1, 28, 28])
Conv1 output size: torch.Size([1, 32, 28, 28])


In [11]:
out = pool(out)
print('Conv1 output size after pooling: {}'.format(out.shape))

Conv1 output size after pooling: torch.Size([1, 32, 14, 14])


In [12]:
out = conv2(out)
print('Conv2 output size: {}'.format(out.shape))

Conv2 output size: torch.Size([1, 64, 14, 14])


In [13]:
out = pool(out)
print('Conv2 output size after pooling: {}'.format(out.shape))

Conv2 output size after pooling: torch.Size([1, 64, 7, 7])


In [14]:
# flatten the tensor while keeping the first dimension (batch dimension)
out = out.view(out.size(0), -1)
print(out.shape)

torch.Size([1, 3136])


Now that the tensor is one dimension (except for batch dimension), this will pass through the fully-connected layer.  
Output layer will be 10 dimensions (MNIST).

In [15]:
fc = nn.Linear(3136, 10)      # input_dim = 3,136, output_dim = 10
out = fc(out)
print(out.shape)

torch.Size([1, 10])


# 2. MNIST classification using CNN

In [16]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# fix the random seed
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [18]:
# set the parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [19]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 168595063.46it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 23362718.19it/s]


Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 78242031.39it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 4683020.84it/s]


Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [20]:
# data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [22]:
# modeling
class CNN(torch.nn.Module):

  def __init__(self):
    super(CNN, self).__init__()   # call __init__ of the parent class

    # first layer
    # ImgIn shape = (?, 1, 28, 28)      # channel 1, 28x28 image
    #   Conv     -> (?, 32, 28, 28)
    #   Pool     -> (?, 32, 14, 14)
    self.layer1 = torch.nn.Sequential(
        torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2))

    # second layer
    # ImgIn shape = (?, 32, 14, 14)
    #   Conv     -> (?, 64, 14, 14)
    #   Pool     -> (?, 64, 7, 7)
    self.layer2 = torch.nn.Sequential(
        torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(kernel_size=2, stride=2))

    # fully-connected layer, 7x7x64 inputs -> 10 outputs
    self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

    # weights initialization
    torch.nn.init.xavier_uniform_(self.fc.weight)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.view(out.size(0), -1)     # flatten the tensor for fc layer
    out = self.fc(out)

    return out

In [23]:
# define CNN model
model = CNN().to(device)

In [24]:
# loss function and optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [26]:
# total # of batches
total_batch = len(data_loader)
print('total # of batches: {}'.format(total_batch))

total # of batches: 600


Each batch size is 100. So the total number of training data is 60,000.

In [28]:
for epoch in range(training_epochs):
  avg_cost = 0

  for X, y in data_loader:
    # image is already size of 28x28, no reshpae necessary
    # label is not one-hot encoded
    X = X.to(device)
    y = y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.220241472
[Epoch:    2] cost = 0.0597546063
[Epoch:    3] cost = 0.0446927845
[Epoch:    4] cost = 0.035970971
[Epoch:    5] cost = 0.0300414059
[Epoch:    6] cost = 0.0246485081
[Epoch:    7] cost = 0.0211722106
[Epoch:    8] cost = 0.0179522522
[Epoch:    9] cost = 0.0152001316
[Epoch:   10] cost = 0.0121086081
[Epoch:   11] cost = 0.0108764404
[Epoch:   12] cost = 0.00928280875
[Epoch:   13] cost = 0.00738868164
[Epoch:   14] cost = 0.00771207549
[Epoch:   15] cost = 0.00769420993


In [29]:
# get test accuracy
with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9842000007629395
